# Множественная диспетчеризация

**Множественная диспетчеризация** является ключевой особенностью Юлии, которую мы рассмотрим в этой записной книжке. Это помогает сделать программное обеспечение быстрым. 

Это также делает программное обеспечение расширяемым, программируемым и совершенно забавным. Это может означать прорыв в параллельных вычислениях.

1. Римские цифры
2. Функции
3. Параллельные вычисления

## 1. Римские числа (просто для веселья)

Давайте определим **новую структуру**, которая представляет римскую цифру. 
Для простоты кодирования мы будем иметь дело только с числами от 0 до 9. 

**Упражнение**: Расширьте это до больших чисел. (Напомним, что римские числа - это система с основанием 10!)

In [ ]:
struct Roman
    n::Int
end

Base.show(io::IO, r::Roman) = print(io, 'ⅰ' + (r.n - 1) % 10 )  # nice display; 'ⅰ' is a Unicode Roman numeral

Мы можем создать объект этого типа следующим образом:

In [ ]:
Roman(4)

In [ ]:
typeof.([5 5.0 Roman(5) "Five" '5'  5//1])

Больше римских цифр!

In [ ]:
x = [7 1 2 5 8 9]
Roman.(x)   # либо map(Roman, x)  or  [Roman(w) for w in x]

Было бы неплохо иметь возможность складывать римские цифры, как обычные числа:

In [ ]:
Roman(4) + Roman(5)

Но Джулия не знает, как это сделать. Давайте научим этому импортом функции `+`, которая затем позволит нам расширить ее определения:

In [ ]:
import Base: +, *

+(a::Roman, b::Roman) = Roman(a.n + b.n)

In [ ]:
Roman(4) + Roman(5)

Это **добавило новый метод** к функции `+`:

In [ ]:
methods(+)

In [ ]:
import Base.*
*(i::Roman, j::Roman) = Roman(i.n * j.n)                     # Умножай как истинный римлянин!

In [ ]:
Roman(3) * Roman(2)

In [ ]:
Roman.(1:3) .* [Roman(1) Roman(2) Roman(3)]

но 

In [ ]:
Roman(3) * 2

In [ ]:
# Complicated mytimes to decide what to do based on type
# not suggested, better way coming soon
function mytimes(i,j)
  if isa(i,Roman) & isa(j,Number)
        return  fill(1, i.n, j)   # i by j matrix with ones
    elseif    isa(i,Number) & isa(j,Roman) 
        return "Δ"^ (i*j.n)   #  i * j deltas
    else
        return("I Don't know")
    end
end

In [ ]:
mytimes(4,Roman(3)) # Twelve 

In [ ]:
mytimes(Roman(4),3) # 4x3 matrix with ones

Самое простое, что можно сделать - это явно определить умножение «римского» на число. Мы можем сделать это так, как считаем нужным:

In [ ]:
*(i::Number, j::Roman) = "😄"^ (i*j.n)        #  i * j happy faces

*(i::Roman, j::Number) =   fill(1, i.n, j)       # i by j matrix

In [ ]:
3 * Roman(3) # Nine happys

In [ ]:
Roman(3) * 5  # Three by Five matrix of ones

In [ ]:
t(x::Roman,y::Roman) = x.n * y.n

In [ ]:
t(Roman(5),Roman(4))

In [ ]:
# Обратите внимание, насколько туго ассемблеру!
@code_native t(Roman(2),Roman(4))

## Функции

In [ ]:
import Base: *, +, ^

In [ ]:
*(α::Number,   g::Function) = x -> α * g(x)   # Скалярное умножение

*(f::Function, λ::Number)   = x -> f(λ * x)   # Масштабировать аргумент

*(f::Function, g::Function) = x -> f(g(x))    # Композиция функций - злоупотребление обозначениями!  use \circ in Julia 0.6

^(f::Function, n::Integer) = n == 1 ? f : f*f^(n-1) # Наивный алгоритм возведения в степень посредством рекурсивного умножения

In [ ]:
+(f::Function, g::Function) = x -> f(x) + g(x)

Например, экспоненциальная функция определяется как

$$\exp(x) = \sum_{n=0}^\infty \frac{1}{n!} x^n.$$

Мы можем думать об этом только с точки зрения функций:

$$\exp = \sum_{n=0}^\infty \frac{1}{n!} \mathrm{pow}_n,$$

где $\mathrm{pow}_n(x) = x^n$.

(начинает размывать символическое с числовым!)

In [ ]:
pow(n) = x -> x^n

myexp = sum(1/factorial(big(n)) * pow(n) for n in 0:100)   # ряды Тейлора не эффективны!

In [ ]:
[myexp(1); exp(1); exp(big(1))]

In [ ]:
f = x -> x^2
f(10)

In [ ]:
g = 3f
g(10)

In [ ]:
(f^2)(10)  # так как мы определили умножение функций как композицию

In [ ]:
using Plots;
gr()

In [ ]:
x = pi*(0:0.001:4)

plot(x, sin.(x),    c="black", label="Fun")
plot!(x, (12*sin).(x),    c="green", label="Num * Fun")
plot!(x, (sin*12).(x),    c="red", alpha=0.9, label="Fun * Num")
plot!(x, (5*sin*exp).(x), c="blue", alpha=0.2, label="Num * Fun * Fun")

In [ ]:
plot([12*sin, sin*12, 5*sin*exp], 0:0.01:4π, α=[1 .9 .2], c=[:green :red :blue])

<img src="https://lh4.googleusercontent.com/--z5eKJbB7sg/UffjL1iAd4I/AAAAAAAABOc/S_wDVyDOBfQ/gauss.jpg">

###  "Sin^2 phi для меня отвратителен, хотя Лаплас использовал это обозначение; следует ли опасаться, что sin^2 phi может стать двусмысленным, что, возможно, никогда не произойдет, или, в наиболее редких случаях, будет восприниматься как о sin(phi^2), ну тогда давайте напишем (sin phi)^2, а не sin^2 phi, что по аналогии должно означать sin(sin phi)." -- Гаусс

In [ ]:
x=(0:0.01:2) * pi;

plot(x, (sin^2).(x), c="blue")     # Квадрат работает, y=sin(sin(x)), Гаусс был бы рад!
plot!(x, sin.(x).^2,  c="red")         

# Упражнение

In [ ]:
h(a, b::Any) = "fallback"
h(a::Number, b::Number) = "a and b are both numbers"
h(a::Number, b) = "a is a number"
h(a, b::Number) = "b is a number"
h(a::Integer, b::Integer) = "a and b are both integers"

In [8]:
# Поиграйтесь с h